# MODELO LSTM

In [ ]:
import pandas as pd
import numpy as np
np.bool = np.bool_
from sklearn.preprocessing import StandardScaler
from gluonts.torch.model.deepar import DeepAREstimator
from gluonts.dataset.common import ListDataset
from gluonts.evaluation.backtest import make_evaluation_predictions

### Procesamiento de Datos

Lectura de los archivos y agrupamiento de ventas por período y producto.

In [ ]:
sells = pd.read_csv("sell-in.txt", sep = "\t")
filter_id = pd.read_csv("productos_a_predecir.txt", sep = "\t")
sells = sells[sells.product_id.isin(filter_id.product_id)]
productos = sells.product_id[sells.periodo == max(sells.periodo)]
productos = np.unique(productos)
df_sells = sells.groupby(["product_id", "periodo"])["tn"].aggregate('sum').reset_index()
df_sells.sort_values(["product_id", "periodo"], inplace = True)
descripcion = pd.read_csv(r"C:\Users\rodri\OneDrive\Documentos\Maestria Ciencia de Datos\Labo 3\DataSets\tb_productos.txt", sep = "\t")
descripcion = descripcion[descripcion.product_id.isin(filter_id.product_id)]

In [ ]:
df_sells.head()

Se crea un dataset con todos los periodos para todos los productos y se realiza un join con el dataset leido.

In [ ]:
df_completo = pd.DataFrame(np.array(np.meshgrid(np.unique(df_sells.product_id), 
                                                np.unique(df_sells.periodo))).T.reshape(-1,2),
                           columns = ["product_id", "periodo"])

df_completo = df_completo.merge(df_sells, how="left", on=["product_id","periodo"])

Se transforma el dataset de formato long a wide.

In [ ]:
df_lstm = df_completo.pivot(columns="product_id", values = "tn", index = "periodo")
df_lstm.columns.name = None

Se utiliza la función interpole para identificar los meses en los que no hubo ventas de productos existentes.

In [ ]:
interpolado = df_lstm.interpolate(axis=0,limit_area="inside",limit_direction="both")
df_lstm.fillna(-1, inplace=True)
df_lstm[(interpolado>0) & (df_lstm == -1)] = 0

Se crea un dataset indicando la antigüedad de cada producto.

In [ ]:
lista_antiguedad = []
nombres_antiguedad = []
for k in df_lstm.columns:
  df_antiguedad = df_lstm.loc[:,k]
  antiguedad = []
  nombre = "antiguedad_" + str(k)
  if sum(df_antiguedad == -1) > 0:
    antiguedad = [-1] * sum(df_antiguedad == -1)
    antiguedad.extend(range(36 - sum(df_antiguedad == -1)))
  if (np.mean(df_antiguedad[24:]) < 0.5 * np.mean(df_antiguedad[12:24])) & (len(antiguedad) == 0):
    antiguedad = [-2] * 36
  if len(antiguedad) == 0:
    antiguedad = [-3] * 36
  
  lista_antiguedad.append(antiguedad)
  nombres_antiguedad.append(nombre)

diccionario = dict(zip(nombres_antiguedad, lista_antiguedad))

df_con_antiguedad = pd.DataFrame(diccionario)
df_con_antiguedad.index = lstm_delta.index


Se vuelven a imputar los meses de los producto que no existían en NAN.

In [ ]:
df_lstm[df_lstm == -1] = np.NaN

#### Escalado de datos

In [ ]:
scaler = StandardScaler()
scaler = scaler.fit(df_lstm)
df_lstm_scale = pd.DataFrame(scaler.transform(df_lstm), columns=df_lstm.columns, index = df_lstm.index)

#### Imputación de NA

Se utiliza el promedio de ventas para cada mes para imputar los NA de productos que no existían.

In [ ]:
for i in np.unique(descripcion["cat1"]) :
  for j in np.unique(descripcion["cat2"]) :
    ids = list(np.unique(descripcion[(descripcion["cat1"] == i) & (descripcion["cat2"] == j)].product_id))
    promedios = pd.DataFrame(np.tile(df_lstm_scale[ids].mean(axis = 1), (len(ids), 1)).transpose(), columns = ids, index=df_lstm_scale.index)
    df_lstm_scale[ids] = df_lstm_scale[ids].fillna(promedios)

#### Imputación de ventas en 201908

Se imputan las ventas de este mes con un interpolación.

In [ ]:
df_lstm_scale.iloc[df_lstm_scale.index == 201908,:] = np.NaN
df_lstm_scale.interpolate(method='polynomial',order = 2, axis=0, inplace=True)

#### Creación dataframe de ventas escalado

In [ ]:
df_lstm_2 = pd.DataFrame(scaler.inverse_transform(df_lstm_scale), columns=df_lstm.columns, index = df_lstm.index)

#### Calculo de pesos a incorporar al modelo

In [ ]:
pesos = list(df_lstm_2.loc[[201901,201902,201903],:].mean(axis = 0))

### Creación de DataFrame con variables delta lag

In [ ]:
columnas = df_lstm.columns

Se crean 12 variables para cada producto por la diferencia entre las ventas de un periodo con el del mes anterior al año anterior.

In [ ]:
def create_lagged_features(data, lag=12):
    list_df = []
    for i in range(1, lag + 1):
        data_lag = data.diff(periods=i, axis=0)
        data_lag.columns = f"delta_{i}_" + columnas.astype(str)
        data_lag.iloc[0:i,:] = data_lag.iloc[12:i+12,:]
        list_df.append(data_lag)
    return pd.concat(list_df, axis=1)

In [ ]:
lstm_delta = create_lagged_features(df_lstm_scale, lag=12)

### Creación de variable roll mean

Se crean variables de roll mean con el objetivo de identificar si se compró mucho o poco en los últimos meses.

In [ ]:
def create_mean_features(data, mean):
    list_df = []
    for i in range(1, mean - 1):
        data_mean = data.rolling(i+1, min_periods=0).sum()
        data_mean.columns = f"mean_{i}_" + columnas.astype(str)
        list_df.append(data_mean)
    return pd.concat(list_df, axis=1)

In [ ]:
lstm_mean = create_mean_features(df_lstm_scale, 6)

### Concatenación de los DataFrames

In [ ]:
df_modelo = pd.concat([df_lstm_scale,lstm_delta,lstm_mean, df_con_antiguedad], axis=1)

#### Modificación del índice en formato datetime

In [ ]:
df_deepar = df_modelo.copy()
df_deepar.index = df_deepar.index.map(str) + "01"
df_deepar.index = pd.to_datetime(df_deepar.index)

## Entrenamiento y predicción del modelo

En el algoritmo DeepAR se envía el dataset completo directamente y se entrena el modelo.

In [ ]:
train_ds = PandasDataset(dict(df_deepar))

estimator = DeepAREstimator(freq='M', prediction_length=2, 
                            num_layers=2, trainer_kwargs={'max_epochs':100}, 
                            hidden_size = 1000)

predictor = estimator.train(train_ds, num_workers=8,one_dim_target=False)

Se generan las predicciones.

In [ ]:
prediccion = predictor.predict(train_ds)

Se graban las predicciones en un CSV.

In [ ]:
predicciones.to_csv("LSTM_completo_1.csv", index = False)
pred_2 = list(prediccion)

El modelo genera predicciones para todas las series de tiempo incorporadas, por lo tanto se deben filtrar las series de ventas de productos.
No se incorporaron variables exógenas al modelo porque se le debían pasar los valores de las variables exógenas en el período a predecir. Por lo tanto se incorporaron todas las variables como series diferentes.

El modelo devuelve un sample de 100 prediciones de la serie en la ventana de tiempo dado. Para calcular la predicción se debe calcular el promedio de este sample. Esto permite conocer el intervalo de confianza de las predicciones realizadas

In [ ]:
all_preds = list()
for item in pred_2:
    family = item.item_id
    p = item.samples.mean(axis=0)[1]
    all_preds += [pd.DataFrame({"product_id": [family], "tn": [p]})]
all_preds = pd.concat(all_preds, ignore_index=True)

Se filtran las predicciones de ventas.

In [ ]:
all_preds = all_preds[all_preds.product_id.isin(productos.astype(str))]

Se ordenan las predicciones.

In [ ]:
all_preds["product_id"] = all_preds.product_id.astype(int)
all_preds.sort_values(by = ["product_id"], inplace = True)

Se revierte el escalado de los valores de las ventas.

In [ ]:
y_final = scaler.inverse_transform(np.array(all_preds.tn).reshape(1,-1))

df_prediccion = pd.DataFrame({"product_id" : all_preds.product_id,"tn" : y_final[0,:]})

Se graban las predicciones de en un CSV.

In [ ]:
df_prediccion.to_csv("deepar_10.csv", index = False)